In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

In [ ]:
df = pd.DataFrame({'Reviews':[], 'Stars':[], 'Title':[]})

In [ ]:
driver = webdriver.Chrome('./chromedriver_win32/chromedriver')

In [ ]:
def scrape_film(url):
    
    driver.get(url)

    reviews = driver.find_elements_by_class_name("audience-reviews__review--mobile.js-review-text.clamp.clamp-4.js-clamp")
    rev = []
    for r in reviews:
        rev.append(r.text)

    ratings = driver.find_elements_by_class_name("star-display")
    stars = []
    for s in ratings:
        html = s.get_attribute('innerHTML')
        star_count = float(html.count("display__filled"))
        if "display__half" in html:
            star_count += 0.5

        stars.append(star_count)

    if len(reviews) != len(stars):
        print("Warning: length of star values and reviews does not match!")

    title_source = driver.find_elements_by_class_name("panel-heading")
    title = title_source[0].text
    if "REVIEWS" in title:
        title = title[:-8]
    
    print(title)

    return pd.DataFrame({"Reviews":rev, "Stars":stars, "Title":title})

In [ ]:
# test obtaining reviews/ratings for one movie ('The Wolf Man'):

scrape_film('https://www.rottentomatoes.com/m/wolf_man/reviews?type=user')

In [ ]:
# Get list of horror titles from the main page

driver.get('https://www.rottentomatoes.com/top/bestofrt/top_100_horror_movies/')

films_elements = driver.find_elements_by_class_name("unstyled.articleLink")
films = []
for e in films_elements:
    films.append(e.text)
films = [f for f in films if f != '']

# Get list of links to 100 horror movie sites
links = []
for i in range(1, 101):
    
    element = driver.find_elements_by_xpath("//*[@id='top_movies_main']/div/table/tbody/tr[" + str(i) + "]/td[3]/a")
    for e in element:
        href = e.get_attribute('href')
        links.append(href)

In [ ]:
# iterate through horror movie Rotten Tomato review links and successively append scraped ratings/reviews to df

count = 1
for link in links:
    print(str(count) + " out of 100")
    reviews_url = link + '/reviews?type=user'
    
    film_df = scrape_film(reviews_url)
    
    df = df.append(film_df)
    
    time.sleep(10) # So our IP address doesn't get blocked by RottenTomatoes
    
    count += 1
print("Done")

In [ ]:
# export our df to CSV
df.to_csv(r'.\reviews_df.csv', index = False)